In [61]:
from torch import nn
import torch
import pandas as pd
import numpy as np
import os
from torchvision.io import read_image
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [49]:
transformer_model = nn.Transformer(batch_first=True)
BATCH_SIZE = 1
ENC_SEQ_LEN = 10
DEC_SEQ_LEN = 1
src = torch.rand((BATCH_SIZE, ENC_SEQ_LEN, transformer_model.d_model))
tgt = torch.rand((BATCH_SIZE, DEC_SEQ_LEN, transformer_model.d_model))
out = transformer_model(src, tgt)
out.flatten(start_dim=1).shape, out.shape

(torch.Size([1, 512]), torch.Size([1, 1, 512]))

In [159]:
encoder_layer = nn.TransformerEncoderLayer(d_model=6, nhead=3, batch_first=True)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
transformer_encoder

TransformerEncoder(
  (layers): ModuleList(
    (0): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=6, out_features=6, bias=True)
      )
      (linear1): Linear(in_features=6, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=6, bias=True)
      (norm1): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=6, out_features=6, bias=True)
      )
      (linear1): Linear(in_features=6, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=6, bias=True)


In [160]:
BATCH_SIZE = 1
ENC_SEQ_LEN = 15
src = torch.rand(BATCH_SIZE, ENC_SEQ_LEN, encoder_layer.self_attn.embed_dim)
out = transformer_encoder(src)
ENC_SEQ_LEN = 16
src = torch.rand(BATCH_SIZE, ENC_SEQ_LEN, encoder_layer.self_attn.embed_dim)
out = transformer_encoder(src)
out.shape, src

(torch.Size([1, 16, 6]),
 tensor([[[0.4311, 0.3375, 0.3744, 0.2338, 0.7313, 0.2425],
          [0.7524, 0.4026, 0.5429, 0.5113, 0.1019, 0.8243],
          [0.3046, 0.9968, 0.8439, 0.9041, 0.5958, 0.5893],
          [0.6594, 0.0069, 0.8752, 0.9730, 0.9796, 0.9051],
          [0.7478, 0.1812, 0.7123, 0.6153, 0.6781, 0.7850],
          [0.4912, 0.2456, 0.5649, 0.1205, 0.6635, 0.7606],
          [0.2108, 0.1277, 0.7877, 0.7445, 0.0157, 0.1731],
          [0.5433, 0.7839, 0.3946, 0.4862, 0.8715, 0.3794],
          [0.6247, 0.5759, 0.0515, 0.8373, 0.3619, 0.7004],
          [0.6730, 0.1387, 0.6259, 0.9951, 0.7938, 0.3173],
          [0.9889, 0.7118, 0.4438, 0.8115, 0.3080, 0.4864],
          [0.9882, 0.4976, 0.1106, 0.1627, 0.4282, 0.8043],
          [0.3383, 0.9685, 0.9702, 0.5511, 0.5124, 0.4251],
          [0.5222, 0.2510, 0.3907, 0.1213, 0.8683, 0.4392],
          [0.1580, 0.2315, 0.6607, 0.4778, 0.1598, 0.1549],
          [0.5270, 0.9304, 0.2287, 0.6299, 0.5448, 0.3195]]]))

In [124]:
data_dir = "indoor_forward_9_snapdragon_with_gt/"
imu_cols=["timestamp", "ang_vel_x", "ang_vel_y", "ang_vel_z", "lin_acc_x", "lin_acc_y", "lin_acc_z"]
imu_df = pd.read_csv(data_dir + "imu.txt", delimiter=' ', skiprows=1, names=imu_cols)
imu_df_nostamp = imu_df.loc[:, imu_df.columns != 'timestamp']
imu_df.index = pd.to_datetime(imu_df["timestamp"], unit='s')
imu_df

,timestamp,ang_vel_x,ang_vel_y,ang_vel_z,lin_acc_x,lin_acc_y,lin_acc_z
timestamp,,,,,,,
2018-10-29 14:20:16.292325120,1.540823e+09,-0.006392,-0.008522,-0.007457,0.052672,-0.158017,9.797023
2018-10-29 14:20:16.294326016,1.540823e+09,-0.005326,-0.006392,-0.006392,0.000000,-0.129286,9.768292
2018-10-29 14:20:16.296323072,1.540823e+09,-0.010653,-0.007457,-0.001065,-0.033519,-0.100556,9.749139
2018-10-29 14:20:16.298322944,1.540823e+09,-0.009587,-0.009587,-0.002131,0.043095,-0.110133,9.725197
2018-10-29 14:20:16.300322048,1.540823e+09,-0.005326,-0.005326,-0.004261,0.047884,-0.134075,9.782658
...,...,...,...,...,...,...,...
2018-10-29 14:21:33.118321920,1.540823e+09,-0.214118,-0.024501,0.415453,-5.956743,-0.397435,7.086801
2018-10-29 14:21:33.120321024,1.540823e+09,-0.238619,0.060720,0.476173,7.110743,1.402995,5.717324
2018-10-29 14:21:33.122322944,1.540823e+09,-0.295078,0.106526,0.288687,-3.969566,-0.181958,16.136837


In [52]:
imu_data = imu_df_nostamp.to_numpy(dtype=np.float32)
imu_data = torch.from_numpy(imu_data)
imu_data

tensor([[-6.3916e-03, -8.5221e-03, -7.4569e-03,  5.2672e-02, -1.5802e-01,
          9.7970e+00],
        [-5.3263e-03, -6.3916e-03, -6.3916e-03,  0.0000e+00, -1.2929e-01,
          9.7683e+00],
        [-1.0653e-02, -7.4569e-03, -1.0653e-03, -3.3519e-02, -1.0056e-01,
          9.7491e+00],
        ...,
        [-2.9508e-01,  1.0653e-01,  2.8869e-01, -3.9696e+00, -1.8196e-01,
          1.6137e+01],
        [-7.8830e-02, -7.3503e-02,  4.6019e-01, -1.6280e+00,  9.0979e-02,
          1.6472e+00],
        [-3.4834e-01,  3.0893e-02,  3.9202e-01,  6.1435e+00,  2.1596e+00,
          9.7204e+00]])

In [60]:
src = imu_data[:ENC_SEQ_LEN].unsqueeze(0)
transformer_encoder(src)

tensor([[[ 0.4793, -0.7749,  1.4578,  0.9232, -1.1373, -0.9481],
         [ 0.6458, -0.7473,  1.6251,  0.4890, -0.7620, -1.2505],
         [ 0.7169, -0.5547,  1.3334,  0.8246, -1.0398, -1.2803],
         [ 0.3889, -0.2781,  1.3305,  0.7418, -0.3609, -1.8223],
         [ 0.5309, -1.1024,  1.6237,  0.6225, -0.6418, -1.0329],
         [ 0.8339, -0.6135,  1.3578,  0.4818, -0.4256, -1.6344],
         [ 0.8776, -0.4995,  1.1355,  0.8479, -0.8777, -1.4838],
         [ 0.8451, -1.3519,  1.4210,  0.5744, -0.7680, -0.7206],
         [ 0.8557, -0.5513,  1.3756,  0.6312, -1.1197, -1.1915],
         [ 0.4572,  0.2107,  1.1180,  0.8811, -1.0470, -1.6199],
         [ 0.4676, -1.5945,  1.2820,  0.9487, -0.3439, -0.7598],
         [ 0.7939, -1.0143,  1.7428,  0.0138, -0.5200, -1.0163],
         [ 0.6511, -0.7525,  1.5113,  0.4291, -0.2672, -1.5717],
         [ 0.9575, -1.3649,  0.7857,  0.9521, -0.0283, -1.3021],
         [ 0.4629, -0.4818,  1.3979,  0.8902, -0.7938, -1.4755],
         [ 0.8163, -0.806

In [157]:
class IMUDataset(Dataset):
    def __init__(self, data_dir, label_file, seq_len):
        imu_cols=["timestamp", "ang_vel_x", "ang_vel_y", "ang_vel_z", "lin_acc_x", "lin_acc_y", "lin_acc_z"]
        self.imu_df = pd.read_csv(data_dir + "imu.txt", delimiter=' ', skiprows=1, names=imu_cols)
        self.imu_df.index = pd.to_datetime(self.imu_df["timestamp"], unit='s')
        imu_df_nostamp = self.imu_df.loc[:, self.imu_df.columns != 'timestamp']
        imu_data = imu_df_nostamp.to_numpy(dtype=np.float32)
        self.imu_data = torch.from_numpy(imu_data)

        label_cols=["timestamp","tx","ty","tz","qx","qy","qz","qw","none"]
        self.labels = pd.read_csv(label_file, delimiter=' ', skiprows=1, names=label_cols)
        self.labels_df = self.labels.iloc[: , :-1]
        self.labels_df["timestamp"] = pd.to_datetime(self.labels_df["timestamp"], unit='s')
        labels_df_nostamp = self.labels_df.loc[:, self.labels_df.columns != 'timestamp']
        self.labels = torch.tensor(labels_df_nostamp.to_numpy(), dtype=torch.float32)

        # self.data_dir = data_dir
        # self.transform = transforms.Compose([
        #     transforms.Normalize(mean=[.0], std=[.25]),
        # ])
        self.seq_len = seq_len

    def __len__(self):
        return self.labels.shape[0] - 1

    def __getitem__(self, idx):
        label_ts = self.labels_df.iloc[idx]["timestamp"]
        # imu_index = self.imu_df.index.get_loc(label_ts, method='nearest')
        # label = self.labels[idx]
        # imu = self.imu_data[imu_index-self.seq_len:imu_index]

        label_ts_end = self.labels_df.iloc[idx+1]["timestamp"]

        imu_index_start = self.imu_df.index.get_loc(label_ts, method='nearest')
        imu_index_end = self.imu_df.index.get_loc(label_ts_end, method='nearest')
        imu = self.imu_data[imu_index_start:imu_index_end]

        return imu, label
ENC_SEQ_LEN = 1
vio_dataset = IMUDataset(data_dir, data_dir + "groundtruth.txt", ENC_SEQ_LEN)
vio_dataset.__getitem__(0), len(vio_dataset)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


((tensor([[ 0.0394, -0.1715, -0.1342, -1.8675, -0.9385,  7.2113]]),
  tensor([ 6.9522e+00,  3.4244e+00, -9.4715e-01,  7.1251e-03, -5.9980e-03,
           8.9050e-01, -4.5490e-01])),
 14400)

In [203]:
ENC_SEQ_LEN = 32
vio_dataset = IMUDataset(data_dir, data_dir + "groundtruth.txt", ENC_SEQ_LEN)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [204]:
data_point = vio_dataset.__getitem__(0)[0].unsqueeze(0)
transformer_encoder(data_point).shape

torch.Size([1, 32, 6])

In [206]:
import torch.nn as nn
import torch.nn.functional as F

class IMUTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=6, nhead=3, batch_first=True)
        transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
        self.encoder = transformer_encoder
        self.fc1 = nn.Linear(in_features=6*ENC_SEQ_LEN, out_features=7)

    def forward(self, x):
        x = F.relu(self.encoder(x))
        # x = self.fc1(x[:,-1,:])
        x = self.fc1(x.flatten(start_dim=1))
        return x

model = IMUTransformer()
model

IMUTransformer(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=6, out_features=6, bias=True)
        )
        (linear1): Linear(in_features=6, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=6, bias=True)
        (norm1): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=6, out_features=6, bias=True)
        )
        (linear1): Linear(in_features=6, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
  

In [208]:
model(data_point)

tensor([[-0.0589, -0.2881,  0.2586,  0.5160, -0.0012,  0.3281,  0.1028]],
       grad_fn=<AddmmBackward0>)

In [209]:
model = model.to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
dataloader = torch.utils.data.DataLoader(vio_dataset, batch_size=16, shuffle=True, num_workers=8)
next(iter(dataloader))[0].shape

torch.Size([16, 32, 6])

In [210]:
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html training loop from here

torch.cuda.empty_cache()

num_epochs = 10
for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)

    for phase in ['train']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0

        # Iterate over data.
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            # labels =  torch.ones_like(labels).to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(vio_dataset)

        print(f'{phase} Loss: {epoch_loss:.4f}')

Epoch 0/9
----------
35.52125549316406
25.786216735839844
23.18031883239746
22.320993423461914
22.12074851989746
17.539819717407227
18.301855087280273
13.122197151184082
12.12604808807373
19.72570037841797
29.761049270629883
19.9506893157959
21.220523834228516
25.01456642150879
20.1151123046875
20.03872299194336
27.475521087646484
26.811931610107422
9.079036712646484
24.223478317260742
23.523714065551758
23.13140106201172
16.113121032714844
25.238595962524414
14.324413299560547
14.139582633972168
17.625995635986328
12.076887130737305
16.461118698120117
12.688921928405762
12.94564437866211
16.034595489501953
7.057498931884766
12.859416961669922
13.376384735107422
8.927509307861328
18.10009765625
12.959074020385742
15.27052116394043
19.59908676147461
11.718914985656738
15.963224411010742
11.39657211303711
14.265795707702637
14.385345458984375
11.069875717163086
9.015377044677734
16.388229370117188
12.739788055419922
12.998178482055664
18.923484802246094
12.6312837600708
11.48336982727050

KeyboardInterrupt: 